# Predicting hospital admission at emergency department triage

**Main statement**
Using machine learning, how might we predict the patient admission into the hospital in the emergency department to provide better information that helps the different stakeholders make better decisions.


## Introduction

Emergency departments (ED) are the hospitals' areas where it is provided treatment for patients with severe illnesses and life-threatening injuries 24 hours a day, 365 days a year. Patients of all ages and a nearly infinite variety of conditions and degrees of illness can require attention in emergency rooms and the approach to treatment in an emergency department can vary dramatically from case to case, even for the same medical condition. Emergency departments are places with high-level pressure and are fast-paced. EDs often become overcrowded due to the variety of complaints and injuries they have to deal with, which can be quite complex. Despite the fact that the majority of patients are discharged from the emergency department, it usually is the primary source of hospital admissions.

Triage is one of the most common tools used to improve ED operational performance. It is utilized to sort incoming patients into groups according to their urgency level. There are several tools/scales developed in order to assist the health provider to categorize the patient. Typically, a nurse does the first evaluation based on various factors, including demographics, chief complaints, and vital signs, in order to carry out triage. Once is triaged, the patient is examined by a physician, who evaluates the patient disposition: admitted or discharged. However, the described process tends to highly depend on the medical provider’s knowledge and experience. 

Advantages of optimizing the process of triage and admission of a patient in the emergency department:

- Patients: better opportunity, safety and satisfaction. Reduce complications, adverse events, patient morbidity and mortality.
- Healthcare providers: support in decision making and not only to emergency professionals, prediction could help primary health services to define if the patient needs to consult the emergency room.
- Hospitals management: support decision-making regarding resources management, patient flow, and healthcare spending. Improve indicators like stay length, readmission rates, and support protocol definitions. 

With hospitals increasing adoption of electronic health records and data warehousing, it has been easier to apply statistical models to improve healthcare processes. Many groups have done predictive models to apply to admission disposition in emergency departments.

Selection of variables
- Based on a review of the literature
- Expert consensus 
- Expected relevance and availability
- Related only to the current attention vs including information about past events and medical history

Statistical models
- Classical statistical multivariable logistic regression.
- Naive Bayes.
- Random forest.
- Gradient boosted decision trees (XGBoost) model. 
- Deep neural network.
- Support vector machines. 
- Cox proportional hazards (CPH) model. 
- Tabu Search (TS).
- T-ADAB integrates Adaboost. 
- AUC Score.
- AUROC.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
from scipy import stats

**Declaration of Use of another Machine Learning Project's Data**

The dataset used in this project was originally collected and published by Hong WS, Haimovich AD, Taylor RA (2018) in their paper, 'Predicting hospital admission at emergency department triage using machine learning.' I will be using this dataset as a starting point for my own data science project, with the aim of further exploring the relationships and patterns that were observed in the original study.

The raw data used in the original study was derived from electronic health records of patient visits to the Yale New Haven Health system and is not publicly available due to the ubiquitous presence of protected health information (PHI). A de-identified, processed dataset of all patient visits included in the models, as well as scripts used for processing and analysis, are available on the Github repository (https://github.com/yaleemmlc/admissionprediction) (10.5281/zenodo.1308993). All other data is available within the paper and its Supporting Information files.

Original paper: Hong WS, Haimovich AD, Taylor RA (2018) Predicting hospital admission at emergency department triage using machine learning. PLoS ONE 13(7): e0201016. https://doi.org/10.1371/journal.pone.0201016

## Dataset description

The data covers:
- 3 emergency departments that belonged to the same hospital system, then they used the same electronic health record and the same Emergency Severity Index.
- level I trauma center with an annual census of approximately 85,000 patients, a community hospital-based department with an annual census of approximately 75,000 patients, and a suburban, free-standing department with an annual census of approximately 30,000 patients.
- March 2013 to July 2017 to ensure a 1-year of historical timeframe from the study start period of March 2014 
- Adult attentions that had clearly described the disposition.
- 972 variables, divided into major categories shown in Table. 
- Data elements were obtained by the researchers from the enterprise data warehouse, using SQL queries to extract relevant raw-data in comma-separated value format. All subsequent processing was done in R. 

|             Variable category            | Number of variables |
|:-------------------------------:|:-------------------:|
| Response variable (Disposition) |          1          |
|           Demographics          |          9          |
|        Triage evaluation        |          13         |
|         Chief complaint         |         200         |
|     Hospital usage statistic    |          4          |
|       Past medical history      |         281         |
|      Outpatient medications     |          48         |
|        Historical vitals        |          28         |
|         Historical labs         |         379         |
|       Imaging / EKG counts      |          9          |
|              Total              |         972         |

**Variables description**

Response variable: The primary response variable was the patient's disposition: admited or discharge.

Demographics: included age, gender, primary language, ethnicity, employment status, insurance status, marital status, and religion. The primary language variable was recoded into a binary split (e.g., English vs. non-English), while the top twelve levels comprising >95% of all visits were retained for the religion variable and all other levels binned to one 'Other' category. All unique levels were retained for other demographic variables.

Triage evaluation: included variables routinely collected at triage, such as the name of presenting hospital, arrival time (month, day, 4-hr bin), arrival method, triage vital signs, and ESI (emergency severity index) level assigned by the triage nurse. Triage vital signs included systolic and diastolic blood pressure, pulse, respiratory rate, oxygen saturation, presence of oxygen device, and temperature. Values beyond physiologic limits were replaced with missing values.

Chief complaint: the top 200 most frequent values, which comprised >90% of all visits, were retained as unique categories and all other values binned into 'Other'.

Hospital usage statistic: This include the number of ED visits within one year, the number of admissions within one year, the disposition of the patient's previous ED visit, and the number of procedures and surgeries listed in the patient's record at the time of encounter.

Past medical history: 281 clinically meaningful categories according the Agency for Healthcare Research and Quality (AHRQ) Clinical Classification Software (CCS), each CCS category became a binary variable with the value 1 if the patient's past medical history contained one or more ICD-9 code belonging in that category and 0 otherwise.

Outpatient medications: binned into 48 therapeutic subgroups (e.g. cardiovascular, analgesics), each corresponding variable representing the number of medications in that subgroup, the medication should be active in the moment of the triage.

Historical vitals: A time-frame of one year from the date of patient encounter was used to calculate historical information, which included vital signs, labs and imaging previously ordered from any of the three EDs. **Historical vital signs were represented by the minimum, maximum, median, and the last recorded value of systolic blood pressure, diastolic blood pressure, pulse, respiratory rate, oxygen saturation, presence of oxygen device, and temperature. Values beyond physiologic limits were replaced with missing values.**

Historical labs: Given the diversity of labs ordered within the ED, the 150 most frequent labs comprising 94% of all orders were extracted then divided into labs with numeric values and those with categorical values. The cutoff of 150 was chosen to include labs ordered commonly enough to be significant in the management of most patients (e.g., Troponin T, BNP, CK, D-Dimer), even if they were not as frequent as routine labs like CBC, BMP, and urinalysis. The minimum, maximum, median, and the last recorded value of each numeric lab were included as features. Categorical labs, which included urinalysis and culture results, were recoded into binary variables with 1 for any positive value (e.g. positive, trace, +, large) and 0 otherwise. Any growth in blood culture was labeled positive as were urine cultures with > 49,000 colonies/mL. The number of tests, the number of positives, and the last recorded value of each categorical lab were included as features.

Imaging and EKG counts: The number of orders were counted for each of the following categories: electrocardiogram (EKG), chest x-ray, other x-ray, echocardiogram, other ultrasound, head CT, other CT, MRI, and all other imaging.


## Exploratory Data Analysis

Load the data

In [2]:
# Load data to google coolab  from drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Load data to google coolab from drive
# del df
# df=pd.read_csv('/content/drive/MyDrive/5v_cleandf.csv', nrows=350000, skiprows=0)
#gc.collect()

In [4]:
# Load the data in Anaconda
df = pd.read_csv('/Users/sbaqueroa/Downloads/5v_cleandf.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'dep_name', 'esi', 'age', 'gender', 'ethnicity', 'race',
       'lang', 'religion', 'maritalstatus',
       ...
       'cc_vaginaldischarge', 'cc_vaginalpain', 'cc_weakness', 'cc_wheezing',
       'cc_withdrawal-alcohol', 'cc_woundcheck', 'cc_woundinfection',
       'cc_woundre-evaluation', 'cc_wristinjury', 'cc_wristpain'],
      dtype='object', length=973)

The dataset looks clean.

In [6]:
# Shape
df.shape

(560486, 973)

The shape: 560.486 rows and 973 columns

In [7]:
# First rows
df.head()

,Unnamed: 0,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,1,B,4.0,40.0,Male,Hispanic or Latino,White or Caucasian,English,None,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,B,4.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,B,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,A,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,A,3.0,84.0,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Last rows
df.tail()

,Unnamed: 0,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
560481,560482,A,2.0,49.0,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560482,560483,A,3.0,50.0,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560483,560484,A,3.0,50.0,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560484,560485,A,3.0,50.0,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560485,560486,A,3.0,50.0,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Cleaning

In [9]:
# Check if the  unique elements is the same as the number of rows
df['Unnamed: 0'].nunique() == df.shape[0]

True

In [10]:
# Quick summary of each column, the datatypes and the number of non-null values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 973 entries, Unnamed: 0 to cc_wristpain
dtypes: float64(590), int64(368), object(15)
memory usage: 4.1+ GB


It is supose that the majority of the columns are already processed as dummy columns where if the condition is present the value is = 1 and if it is not present value = 0, df.info( ) describe a high number of columns with float and integer data types, then it is likely that those columns have a wrong data type, but not all of them, for example, it sholud be numeric columns: triage vital signs, hospital usage, number of medications in each subgroup of medication, number of test and number of images done to each patient.

How can I verify each column data type?
- I have the list with the name of all the columns and the category that belong.
- Could I do a loop defining a regex condition that help me to cacth and correct them faster, for example chif complains columns start with the id 'cc_'.
- I can correct the columns data types after completed the missing data verification, because it is possible I have to delete some columns in that step.

Lets see how looks the numeric columns:

In [11]:
# See the characteristics of Numeric columns
df.describe().T # correct data types 

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,560486.0,280243.500000,161798.515825,1.0,140122.25,280243.5,420364.75,560486.0
esi,558029.0,3.012385,0.869106,1.0,2.00,3.0,4.00,5.0
age,560475.0,49.876765,20.417029,18.0,32.00,49.0,64.00,108.0
2ndarymalig,560486.0,0.002155,0.046375,0.0,0.00,0.0,0.00,1.0
abdomhernia,560486.0,0.030634,0.172324,0.0,0.00,0.0,0.00,1.0
...,...,...,...,...,...,...,...,...
cc_woundcheck,557901.0,0.004219,0.064820,0.0,0.00,0.0,0.00,1.0
cc_woundinfection,557901.0,0.002863,0.053426,0.0,0.00,0.0,0.00,1.0
cc_woundre-evaluation,557901.0,0.001233,0.035095,0.0,0.00,0.0,0.00,1.0
cc_wristinjury,557901.0,0.001393,0.037293,0.0,0.00,0.0,0.00,1.0


As it was suspected the columns with binary values are defined as numeric columns.

At first glance highligth 'esi' column because it is a categorical column, that column is classifying the patient in 5 different categories depending on the severity of the clinical situation.

cc_wristpain seems to have a typo because the max number is 2 and this column is binary.

As I said, I need to correct datatypes and now I know I need to verify possible typos.

Lets see how look the categorical columns:

In [12]:
# Non numeric columns
df.select_dtypes('object').head() 

,dep_name,gender,ethnicity,race,lang,religion,maritalstatus,employstatus,insurance_status,disposition,arrivalmode,arrivalmonth,arrivalday,arrivalhour_bin,previousdispo
0,B,Male,Hispanic or Latino,White or Caucasian,English,None,Single,Full Time,Other,Discharge,Walk-in,June,Tuesday,23-02,No previous dispo
1,B,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,Car,January,Tuesday,15-18,No previous dispo
2,B,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,Walk-in,July,Thursday,11-14,Discharge
3,A,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,Car,July,Saturday,11-14,Discharge
4,A,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired,Medicare,Admit,Walk-in,November,Tuesday,07-10,Discharge


Okey, this columns look good, they are specially demographic information, I need to define if I want to include all of them and also how I want to make dummy columns from them.

In [13]:
# Columns names: remember it has 973 columns.
# df.columns.values

Look for missing values

In [14]:
# Duplicated columns, sum the resultant array to get the total number of duplicate rows
df.duplicated(keep=False).sum()

0

There is not duplicated columns

Lets see the rows

In [15]:
# NaN or Null Values
df.isna().sum(axis=0)

Unnamed: 0                  0
dep_name                    0
esi                      2457
age                        11
gender                      0
                         ... 
cc_woundcheck            2585
cc_woundinfection        2585
cc_woundre-evaluation    2585
cc_wristinjury           2585
cc_wristpain             2585
Length: 973, dtype: int64

In [16]:
df.isna().sum().sum()

161875061

Lets try to undesthand the missing data and see if the description of the dataset is coherent with the real data, remember the owners of the data explained that in some of the columns when they found values beyond physiologic limits those values were replaced with missing values.

Additionally, it is important to know that not all the exams and meassures are done in all the patients, it depends on the medical situation and the medical provider consideration.

Tabulate proportion of NA per column. The output is again truncated to only include features with more than ___% missing values.

In [17]:
# Tabulate columns with more than 5% of missing values
truncate = df.isna().mean() > .05
pd.DataFrame({'sum': df.loc[:, truncate].isna().sum(), 'mean': df.loc[:, truncate].isna().mean()}).round(3).sort_values(by = 'mean', ascending = False)

,sum,mean
"phencyclidine(pcp)screen,urine,noconf._min",560486,1.000
"phencyclidine(pcp)screen,urine,noconf._median",560486,1.000
epithelialcells_last,560478,1.000
epithelialcells_min,560455,1.000
epithelialcells_median,560455,1.000
...,...,...
triage_vital_temp,182985,0.326
triage_vital_rr,170487,0.304
triage_vital_dbp,168063,0.300
triage_vital_sbp,167904,0.300


In [18]:
# Tabulate columns with more than 50% of missing values
truncate = df.isna().mean() > .50
pd.DataFrame({'sum': df.loc[:, truncate].isna().sum(), 'mean': df.loc[:, truncate].isna().mean()}).round(3).sort_values(by = 'mean', ascending = False)

,sum,mean
"benzodiazepinesscreen,urine,noconf._median",560472,1.000
"phencyclidine(pcp)screen,urine,noconf._median",560486,1.000
epithelialcells_max,560455,1.000
"phencyclidine(pcp)screen,urine,noconf._last",560486,1.000
"benzodiazepinesscreen,urine,noconf._max",560472,1.000
...,...,...
mchc_min,310702,0.554
mch_min,310703,0.554
mch_median,310703,0.554
mchc_median,310702,0.554


Roughly 400 columns contain at least 5% of missing data. 

Lets visualize the missing data to try to see in which category are located:

In [19]:
# Library that provides a small toolset of flexible and easy-to-use missing data visualizations and utilities that allows you to get a quick visual summary of the completeness 
%matplotlib inline
import missingno as msno

In [20]:
# Create a bar chart to visualize the missing values in each column
# msno.bar(df)

the dataset contains 2 clusters with columns with high number of missing values. I supose my main problem with missing values are with laboratories because not all the lab exams and vital sign meassures.

**Missing values matrix** provides a nullity matrix to examine missing data by row. 
It is ideal to visually check whether the missing values follow any pattern. 
The spark line on the far right marks the most and least complete case.

In [21]:
# Loop to visualize the matrix of missing values per sub_sets of 100 columns

#for i in range(0, 972, 100):
 #   subset_df = df.iloc[:, i:i+100]
 #   msno.matrix(subset_df)

Lets try to see the labels of the columns with a lot of missing data:

In [22]:
# Biggest cluster
# columnss = df.iloc[0:1,410:650].columns.values
# columnss

In [23]:
# Smallest cluster
# columnss = df.iloc[0:1,670:720].columns.values
# columnss

The columns with high number of missing values are all of some of the laboratory and vital signs columns.



**OPTIONS THAT I HAVE TO MANAGE MISSING VALUES:**

**Deletion**
    
   Deletion is when we simply remove data that is missing.
   - *List-wise deletion* is simply the process of deleting rows of data that have missing data in any of the columns. A downside here is that we lose the data for this row which was not missing.
   - *Pairwise deletion* is the process of excluding the missing values selectively when carrying out statistical analysis. For example, if we had 5 rows for a particular column, and 1 of the rows had a `NaN`. When calculating the average, pairwise deletion would simply ignore this row and calculate the average as if the column only had 4 rows. The issue here is that its hard to compare between columns which have had statistics calculated on different numbers of rows.
   - *Column deletion* is the process of removing entire columns. This is usually employed when we have a very high percentage of missing data in an individual column.
   
**Imputation**
  
  Imputation is the process of filling in the missing values with numbers based on our expectation of what the true values for the missing values would be. This is a very deep area with numerous methods that can be applied. 
  
  For now, lets focus on two simple methods:
   - *Mean imputation* is the process of filling in missing values with the average of the non-missing data from that column.
   - *Median imputation* is the process of filling in the missing values with the median (mid-point) of the non-missing data from that column.

**General rules of thumb for imputation**

There is not a single approach to imputation that you can take that applies to all datasets. You will need to tailor your approach to the particular situation you find yourself in. With that being said, there are some rules of thumb which you will be to apply to the majority of cases.

- If the missing data makes up less than 3% of the total observations in a column, your approach will not make much of a difference.

- Depending on the data, 50% of missing data in a column can be a good threshold to remove the column entirely, sometimes it may be less. Consider the following:

    - Is the mean/median calculated from only <=50% of the column a good representation of the true mean/median?   
    - You will be skewing the distribution too heavily. If half your column was missing, then 50% of the post-imputation values would be exactly the same!


- If the goal of your analysis is prediction, try various strategies and compare results. There is nothing preventing you from creating different versions of your dataset. You can apply various techniques, observe the results, and proceed with the one that leads to the best performance.

### Set correct datatypes

Split the data in categorical and numeric dataframes

Start a new dataset with categorical columns 

In [24]:
categorical_df = df.select_dtypes('object').copy()

In [25]:
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dep_name          560486 non-null  object
 1   gender            560486 non-null  object
 2   ethnicity         560486 non-null  object
 3   race              560460 non-null  object
 4   lang              560486 non-null  object
 5   religion          560486 non-null  object
 6   maritalstatus     560486 non-null  object
 7   employstatus      560486 non-null  object
 8   insurance_status  560486 non-null  object
 9   disposition       560486 non-null  object
 10  arrivalmode       538602 non-null  object
 11  arrivalmonth      560486 non-null  object
 12  arrivalday        560486 non-null  object
 13  arrivalhour_bin   560486 non-null  object
 14  previousdispo     560486 non-null  object
dtypes: object(15)
memory usage: 64.1+ MB


In [26]:
numeric_df = df.select_dtypes('number').copy()

In [27]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 958 entries, Unnamed: 0 to cc_wristpain
dtypes: float64(590), int64(368)
memory usage: 4.0 GB


In [28]:
#sanity check that we have all the columns
df.shape[1] == numeric_df.shape[1] + categorical_df.shape[1]

True

Before starting to process the columns it is necessary set the correct datatypes in some numeric_df columns.

In [29]:
numeric_df.describe()

,Unnamed: 0,esi,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
count,560486.000000,558029.000000,560475.000000,560486.000000,560486.000000,560486.000000,560486.000000,560486.000000,560486.000000,560486.000000,...,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000,557901.000000
mean,280243.500000,3.012385,49.876765,0.002155,0.030634,0.014807,0.000023,0.005179,0.009968,0.051054,...,0.001298,0.000717,0.013085,0.001052,0.001165,0.004219,0.002863,0.001233,0.001393,0.002518
std,161798.515825,0.869106,20.417029,0.046375,0.172324,0.120779,0.004816,0.071782,0.099342,0.220108,...,0.036001,0.026900,0.113638,0.032475,0.034113,0.064820,0.053426,0.035095,0.037293,0.050156
min,1.000000,1.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,140122.250000,2.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,280243.500000,3.000000,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,420364.750000,4.000000,64.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,560486.000000,5.000000,108.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


First,'esi column' is a categorical column. The rest of columns will be corrected by group acording with the categories.

In [30]:
# Change 'esi' datatype
numeric_df['esi'] = numeric_df['esi'].astype('object')

In [31]:
# Add 'esi column' to the categorical_df
categorical_df = pd.concat([categorical_df, numeric_df['esi']], axis=1) 

In [32]:
# Sanity check
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dep_name          560486 non-null  object
 1   gender            560486 non-null  object
 2   ethnicity         560486 non-null  object
 3   race              560460 non-null  object
 4   lang              560486 non-null  object
 5   religion          560486 non-null  object
 6   maritalstatus     560486 non-null  object
 7   employstatus      560486 non-null  object
 8   insurance_status  560486 non-null  object
 9   disposition       560486 non-null  object
 10  arrivalmode       538602 non-null  object
 11  arrivalmonth      560486 non-null  object
 12  arrivalday        560486 non-null  object
 13  arrivalhour_bin   560486 non-null  object
 14  previousdispo     560486 non-null  object
 15  esi               558029 non-null  object
dtypes: object(16)
memory usage: 68.4+ MB


In [33]:
# Bookkeeping
numeric_df.drop(columns='esi', inplace=True)
numeric_df.head()

,Unnamed: 0,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,1,40.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,66.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,66.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,66.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,84.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# Bookkeeping, drop Unnamed:0 column because it is not necessary, the index was vefified before.
numeric_df.drop(columns='Unnamed: 0', inplace=True)
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,40.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,66.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,66.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,84.0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now I can group columns per category a set them in the correct dataframe according with the correct datatype

|             Variable category            | Number of variables |
|:-------------------------------:|:-------------------:|
| Response variable (Disposition) |          1          |
|           Demographics          |          9          |
|        Triage evaluation        |          13         |
|         Chief complaint         |         200         |
|     Hospital usage statistic    |          4          |
|       Past medical history      |         281         |
|      Outpatient medications     |          48         |
|        Historical vitals        |          28         |
|         Historical labs         |         379         |
|       Imaging / EKG counts      |          9          |
|              Total              |         972         | 

Chief complains columns:

In [35]:
# Lets see the chief complain columns (categorical): the top 200 most frequent values (1=chief presented 0=chief no presented)

chief_complain_df = numeric_df.filter(regex='^cc_').astype('object').copy()
chief_complain_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 200 entries, cc_abdominalcramping to cc_wristpain
dtypes: object(200)
memory usage: 855.2+ MB


Chief_complain_df is already processed then I am not going to concatenate with categorical_df, I will wait and subsequently concatenate to the processed_df.

In [36]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex='^cc_'), inplace=True)

In [37]:
# Sanity check
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 756 entries, age to n_surgeries
dtypes: float64(389), int64(367)
memory usage: 3.2 GB


Medication columns:

In [38]:
# Lets see the outpatient medications columns: binned into 48 therapeutic subgroups (e.g. cardiovascular, analgesics), each corresponding variable representing the number of medications in that subgroup.  

medication_df = numeric_df.filter(regex='^meds_').copy()
medication_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 48 columns):
 #   Column                                              Non-Null Count   Dtype
---  ------                                              --------------   -----
 0   meds_analgesicandantihistaminecombination           560486 non-null  int64
 1   meds_analgesics                                     560486 non-null  int64
 2   meds_anesthetics                                    560486 non-null  int64
 3   meds_anti-obesitydrugs                              560486 non-null  int64
 4   meds_antiallergy                                    560486 non-null  int64
 5   meds_antiarthritics                                 560486 non-null  int64
 6   meds_antiasthmatics                                 560486 non-null  int64
 7   meds_antibiotics                                    560486 non-null  int64
 8   meds_anticoagulants                                 560486 non-null  int64
 9   meds

In [39]:
medication_df.isna().sum().sum()

0

medication_df is already processed but I am going to drop those columns in the numeric_df and I will wait and subsequently concatenate to the processed_df.

In [40]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex='^meds_'), inplace=True)

In [41]:
# Sanity check
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,cxr_count,echo_count,ekg_count,headct_count,mri_count,otherct_count,otherimg_count,otherus_count,otherxr_count,n_surgeries
0,40.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,66.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,66.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,66.0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,2
4,84.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5


Imaging columns:

In [42]:
# Lets see the imaging columns: number of images order per category

imaging_df = numeric_df.filter(regex='^.*_count$').copy()
imaging_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype
---  ------                      --------------   -----
 0   bloodua_count               560486 non-null  int64
 1   glucoseua_count             560486 non-null  int64
 2   ketonesua_count             560486 non-null  int64
 3   leukocytesua_count          560486 non-null  int64
 4   nitriteua_count             560486 non-null  int64
 5   pregtestur_count            560486 non-null  int64
 6   proteinua_count             560486 non-null  int64
 7   bloodculture,routine_count  560486 non-null  int64
 8   urineculture,routine_count  560486 non-null  int64
 9   cxr_count                   560486 non-null  int64
 10  echo_count                  560486 non-null  int64
 11  ekg_count                   560486 non-null  int64
 12  headct_count                560486 non-null  int64
 13  mri_count                   560486 non-null 

In [43]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex='^.*_count$'), inplace=True)

In [44]:
# Sanity check
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,dbp_max,o2_device_max,pulse_median,resp_median,spo2_median,temp_median,sbp_median,dbp_median,o2_device_median,n_surgeries
0,40.0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,66.0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,66.0,0,0,0,0,0,0,0,0,0,...,77.0,0.0,83.0,16.0,NaN,98.00,125.0,77.0,0.0,2
3,66.0,0,0,0,0,0,0,0,0,0,...,83.0,0.0,74.5,16.0,98.0,97.85,132.5,77.5,0.0,2
4,84.0,0,0,0,0,0,0,0,0,0,...,76.0,0.0,75.0,18.0,97.0,98.20,132.0,70.0,0.0,5


Historical vitals and historical labs

In [45]:
hist_vit_lab_df = numeric_df.filter(regex=('^.*_last$|^.*_min$|^.*_max$|^.*_median$|^.*_npos')).copy()
hist_vit_lab_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 398 entries, absolutelymphocytecount_last to o2_device_median
dtypes: float64(380), int64(18)
memory usage: 1.7 GB


In [46]:
hist_vit_lab_df.isna().sum()

absolutelymphocytecount_last         342560
acetonebld_last                      560090
alanineaminotransferase(alt)_last    434832
albumin_last                         448952
alkphos_last                         433755
                                      ...  
spo2_median                          232899
temp_median                          214260
sbp_median                           212611
dbp_median                           212631
o2_device_median                     218444
Length: 398, dtype: int64

In [80]:
# msno.bar(hist_vit_lab_df)

This columns contain high number of missing values, however there are some of them that look without missig values, I suspect those columns are the columns with integer datatype, lets see:

In [48]:
# Extract the columns without missing values:
int_columns = hist_vit_lab_df.select_dtypes(include=['int']).columns
int_columns

Index(['bloodua_last', 'glucoseua_last', 'ketonesua_last', 'leukocytesua_last',
       'nitriteua_last', 'pregtestur_last', 'proteinua_last',
       'bloodculture,routine_last', 'urineculture,routine_last',
       'bloodua_npos', 'glucoseua_npos', 'ketonesua_npos', 'leukocytesua_npos',
       'nitriteua_npos', 'pregtestur_npos', 'proteinua_npos',
       'bloodculture,routine_npos', 'urineculture,routine_npos'],
      dtype='object')

Evaluate if those columns contain valuable information in terms of number of a last positive result in the paraclinic.

In [49]:
# Count the number of 1s in those columns:
cols_to_count = ['bloodua_last', 'glucoseua_last', 'ketonesua_last', 'leukocytesua_last',
                 'nitriteua_last', 'pregtestur_last', 'proteinua_last',
                 'bloodculture,routine_last', 'urineculture,routine_last']
count_ones = (hist_vit_lab_df[cols_to_count] == 1).sum()
print(count_ones)

bloodua_last                 39089
glucoseua_last               13037
ketonesua_last               10272
leukocytesua_last            40880
nitriteua_last                7948
pregtestur_last                 13
proteinua_last               43903
bloodculture,routine_last        1
urineculture,routine_last     2646
dtype: int64


The data in these columns is irrelevant and could cause overfitting in future steps. It will not be take in consideration.

In conclusion, Historical vitals and historical labs columns will be removed because they have a high percentage of missing values and low number of relevant values.

In [50]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex=('^.*_last$|^.*_min$|^.*_max$|^.*_median$|^.*_npos')), inplace=True)

In [51]:
# Sanity check
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,n_edvisits,n_admissions,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_o2,triage_vital_o2_device,triage_vital_temp,n_surgeries
0,40.0,0,0,0,0,0,0,0,0,0,...,0,0,63.0,146.0,85.0,18.0,97.0,0.0,97.0,1
1,66.0,0,0,0,0,0,0,0,0,0,...,0,0,83.0,125.0,77.0,16.0,NaN,0.0,98.0,2
2,66.0,0,0,0,0,0,0,0,0,0,...,1,0,78.0,134.0,78.0,16.0,97.0,NaN,97.8,2
3,66.0,0,0,0,0,0,0,0,0,0,...,2,0,98.0,171.0,92.0,18.0,98.0,0.0,NaN,2
4,84.0,0,0,0,0,0,0,0,0,0,...,1,0,101.0,133.0,72.0,18.0,97.0,0.0,98.4,5


Hospital usage columns:

In [52]:
# Hospital usage statistic
h_usage_df = numeric_df.filter(regex='^n_').copy()
h_usage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   n_edvisits    560486 non-null  int64
 1   n_admissions  560486 non-null  int64
 2   n_surgeries   560486 non-null  int64
dtypes: int64(3)
memory usage: 12.8 MB


In [53]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex='^n_'), inplace=True)
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,varicosevn,viralinfect,whtblooddx,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_o2,triage_vital_o2_device,triage_vital_temp
0,40.0,0,0,0,0,0,0,0,0,0,...,0,0,0,63.0,146.0,85.0,18.0,97.0,0.0,97.0
1,66.0,0,0,0,0,0,0,0,0,0,...,0,1,0,83.0,125.0,77.0,16.0,NaN,0.0,98.0
2,66.0,0,0,0,0,0,0,0,0,0,...,0,1,0,78.0,134.0,78.0,16.0,97.0,NaN,97.8
3,66.0,0,0,0,0,0,0,0,0,0,...,0,1,0,98.0,171.0,92.0,18.0,98.0,0.0,NaN
4,84.0,0,0,0,0,0,0,0,0,0,...,0,0,0,101.0,133.0,72.0,18.0,97.0,0.0,98.4


Triage vitals variables:

In [54]:
# triage vitals
triage_v = numeric_df.filter(regex='^triage_vital_').copy()
triage_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   triage_vital_hr         394434 non-null  float64
 1   triage_vital_sbp        392582 non-null  float64
 2   triage_vital_dbp        392423 non-null  float64
 3   triage_vital_rr         389999 non-null  float64
 4   triage_vital_o2         289169 non-null  float64
 5   triage_vital_o2_device  333631 non-null  float64
 6   triage_vital_temp       377501 non-null  float64
dtypes: float64(7)
memory usage: 29.9 MB


In [55]:
triage_v

,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_o2,triage_vital_o2_device,triage_vital_temp
0,63.0,146.0,85.0,18.0,97.0,0.0,97.0
1,83.0,125.0,77.0,16.0,NaN,0.0,98.0
2,78.0,134.0,78.0,16.0,97.0,NaN,97.8
3,98.0,171.0,92.0,18.0,98.0,0.0,NaN
4,101.0,133.0,72.0,18.0,97.0,0.0,98.4
...,...,...,...,...,...,...,...
560481,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560482,89.0,125.0,82.0,18.0,94.0,0.0,97.5
560483,89.0,135.0,92.0,18.0,98.0,0.0,97.5
560484,77.0,118.0,73.0,16.0,94.0,0.0,98.0


**triage_v has a high number of missing values, I have two options:**

- I reduce the number of rows and keep the information about vital signs in triage
- I drop these columns and keep a higher number of rows

I compared the difference between patients admitted and discharged in a dataset with triage_v and without them. Since it is better to have a balanced relationship between the classes to be predicted than more columns in the model I will exclude the triage_vitals.

In [56]:
# Bookkeeping
numeric_df.drop(columns=numeric_df.filter(regex='^triage_vital_'), inplace=True)
numeric_df.head()

,age,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,...,ulcerskin,umbilcord,unclassified,urinstone,urinyorgca,uteruscancr,uti,varicosevn,viralinfect,whtblooddx
0,40.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,66.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,66.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,84.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It is remaining 'age' column and 281 columns more, 281 is the coincident number of Past Medical History columns.
- Extract the 'age' column
- Set the datatype of past medical history columns to 'object' type.

In [57]:
# Extract 'age' column from numeric_df 
age_df = numeric_df['age'].copy()
age_df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 560486 entries, 0 to 560485
Series name: age
Non-Null Count   Dtype  
--------------   -----  
560475 non-null  float64
dtypes: float64(1)
memory usage: 4.3 MB


In [58]:
# Bookkeeping
numeric_df.drop(columns='age', inplace=True)
numeric_df.head()

,2ndarymalig,abdomhernia,abdomnlpain,abortcompl,acqfootdef,acrenlfail,acutecvd,acutemi,acutphanm,adjustmentdisorders,...,ulcerskin,umbilcord,unclassified,urinstone,urinyorgca,uteruscancr,uti,varicosevn,viralinfect,whtblooddx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# Set the datatype of past medical history columns to 'object' type.
past_md_h_df = numeric_df.astype('object')
past_md_h_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 281 entries, 2ndarymalig to whtblooddx
dtypes: object(281)
memory usage: 1.2+ GB


**Summary:** 
- The datatypes were fixed.
- Columns related to triage vitals, vitals history and laboratories were excluded.
- Currently, the information is fragmented in the following dataframes:

|   **Dataframe**   | **N columns** |                  **Notes**                 |
|:-----------------:|:-------------:|:------------------------------------------:|
|   categorical_df  |       16      | categorical data pending set dummy columns |
| chief_complain_df |      200      |     categorical data, dummy columns ok     |
|   medication_df   |       48      |                numeric data                |
|     imaging_df    |       18      |                numeric data                |
|     h_usage_df    |       3       |                numeric data                |
|       age_df      |       1       |                numeric data                |
|    past_md_h_df   |      281      |     categorical data, dummy columns ok     |
|       Total       |      567      |                                            |

Concatenate the dataframes again to evaluate the number of missing values and clean it.

In [60]:
# Lets concat my provisional dataframe with datatypes corrected to use it in Tableau

pre_cleaned_df = pd.concat([categorical_df, age_df, h_usage_df, chief_complain_df, medication_df, imaging_df, past_md_h_df], axis=1)

In [61]:
# Sanity check:
pre_cleaned_df.shape

(560486, 567)

Missing values

In [64]:
# Sanity check
pre_cleaned_df.isna().sum().sum()

541378

In [ ]:
# Create a bar chart to visualize the missing values in each column
#msno.bar(pre_cleaned_df)

In [65]:
# Lets drop the missing values in my new dataframe (keeping traige_vitals columns)

cleaned_df = pre_cleaned_df.dropna().copy()

In [66]:
# Sanity check
cleaned_df.shape

(534730, 567)

In [67]:
cleaned_df.isna().sum().sum()

0

In [ ]:
# Create a new csv file with my provisional_df

# cleaned_df.to_csv('new.csv', index=True)

## Process the columns

In [81]:
cleaned_df

,dep_name,gender,ethnicity,race,lang,religion,maritalstatus,employstatus,insurance_status,disposition,...,ulcerskin,umbilcord,unclassified,urinstone,urinyorgca,uteruscancr,uti,varicosevn,viralinfect,whtblooddx
0,B,Male,Hispanic or Latino,White or Caucasian,English,None,Single,Full Time,Other,Discharge,...,0,0,0,0,0,0,0,0,0,0
1,B,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,...,0,0,0,0,0,0,0,0,1,0
2,B,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,...,0,0,0,0,0,0,0,0,1,0
3,A,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,Commercial,Discharge,...,0,0,0,0,0,0,0,0,1,0
4,A,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired,Medicare,Admit,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560481,A,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,Disabled,Medicare,Admit,...,0,0,1,0,0,0,0,0,0,0
560482,A,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,Disabled,Medicare,Admit,...,0,0,1,0,0,0,0,0,0,0
560483,A,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,Disabled,Medicare,Discharge,...,0,0,1,0,0,0,0,0,0,0
560484,A,Male,Non-Hispanic,White or Caucasian,English,Episcopal,Single,Disabled,Medicare,Admit,...,0,0,1,0,0,0,0,0,0,0


## Visualize the data

In [ ]:
for column in cleaned_df.select_dtypes("object"):
    cleaned_df[column].value_counts(normalize=True, dropna=False).plot(kind="barh")
    plt.title(f"Relative Frequency of Value - {column}")
    plt.show()

In [ ]:
for column in cleaned_df.select_dtypes("number").columns:

    plt.hist(cleaned_df[column], bins=80)

    plt.axvline(cleaned_df[column].mean(), 0, 1000, label="mean", color="lightcoral")
    plt.axvline(cleaned_df[column].median(), 0, 1000, label="median", color="lightcoral", linestyle="--")

    plt.title(f"Distribution of {column}")
    sns.despine()
    plt.legend()
    plt.show()

In [ ]:
plt.hist(cleaned_df["esi"], bins=20)

plt.axvline(cleaned_df["esi"].mean(), 0, 1000, label="mean", color="lightcoral")
plt.axvline(cleaned_df["esi"].median(), 0, 1000, label="median", color="lightcoral", linestyle="--")

plt.title("Distribution of esi")
sns.despine()
plt.legend()
plt.show()

In [ ]:
sns.boxplot(data=cleaned_df, x = "age", y="disposition")
plt.show()

### Basic Data Analysis

Now that we have a clean dataset we can begin analyzing it to begin developing some insights about the data itself. 

The key tool at this stage are visualizations. We can create useful visuals  to help us learn more about our data. In particular we want to try and cover the following points/questions:  

1. Identify variables which are useful and relevant to what we want to study. 
2. Identify trends and relationships between variables that help us answer our question(s).
3. Do the answers to the above lead to any other areas of investigation? 

It should be noted that there isn't one correct process to follow. However if you cover the above points you are in a good position to begin more detailed analysis and modeling tasks later on.


Explore metrics of interest, what is the data telling, uncover relationships and properties, visualizations

In [ ]:
# Categorical columns
categorical_df = cleaned_df.select_dtypes("object").copy()
categorical_df.head()

In [ ]:
# numeric columns
numeric_df = cleaned_df.select_dtypes("number").copy()
numeric_df.head()

In [ ]:
sns.pairplot(triage_v)
plt.show()

In [ ]:
# Pearson Coefficient
numeric_df.corr(triage_v)

In [ ]:
sns.heatmap(triage_v.corr(), annot=True, vmin=-1, vmax=1, cmap="coolwarm")
plt.show(triage_v)

##### A couple of relevant insights that these graphs reveal are:
-
-
-
-

## Advance Data Analysis

Remember make the prior assumptions

Advance Machine learning techniques 

In [ ]:
from sklearn.linear_model import LogisticRegression

# Notes

**TO ASK**

ABOUT CLEANING

- Create the dummy columns.
- Investigate how reduce the number of columns before modeling or how machine learning can identify wich columns are generating better predictions.

ABOUT MODELING
- How reduce the number of columns, if one column does not have 1s (ones) should I delete it?


**TO DO:**

- Apply the scikit learn library to build the logistic regression model. 
